In [ ]:
import urllib.request as urllib
import pandas as pd
import time
import numpy as np

import os
os.chdir('E:/Research/Processing Datasets')

#Extract countries available in pollutant release dataset
facilities = pd.read_csv('E:\Research\Datasets\Pollutant emissions data\E-PRTR_database_v16_csv\dbo.PUBLISH_FACILITYREPORT.csv', encoding = "ISO-8859-1")
countryCodes = set(facilities['CountryCode'])

#Find common pollutants reported by all countries
#Checked download urls present in text file downloaded through e-reporting API
data = []
myfile = open("dataset _download_links.txt", "r")
lines = myfile.read().split('\n')
#For every record, add new item in list
for index,line in enumerate(lines):
    name = line.split('/')[-1]
    if name[0:2] in countryCodes:
        data.append({"Number": index, "Country": name[0:2], "Pollutant":name.split('_')[1]})
        
myfile.close()
#Drop dups and group counts
df1 = pd.DataFrame(data).drop_duplicates(['Country','Pollutant'], keep= 'last')
df2 = pd.DataFrame(columns = ['No_Reporting_Countries'])
df2['No_Reporting_Countries'] = df1.groupby('Pollutant').apply(lambda x: len(x['Country'].unique()))

#Filter pollutants reported by all countries
reported_pols = df2[df2['No_Reporting_Countries'] == 30]
polNames = pd.DataFrame([{'Pollutant':'1', 'Pollutant_Name':'SO2'},{'Pollutant':'10', 'Pollutant_Name':'CO'},{'Pollutant':'20', 'Pollutant_Name':'C6H6'},{'Pollutant':'5', 'Pollutant_Name':'PM10'},{'Pollutant':'6001', 'Pollutant_Name':'PM2.5'},{'Pollutant':'7', 'Pollutant_Name':'O3'},{'Pollutant':'8', 'Pollutant_Name':'NO2'},{'Pollutant':'9', 'Pollutant_Name':'Nox as No2'}])
reported_pols = pd.merge(reported_pols, polNames, on='Pollutant')
reported_pols[['Pollutant_Name', 'No_Reporting_Countries']]

#From all countries and all polutants datasets urls, filter out and save urls for above pollutants and countries
myfile = open("dataset _download_links.txt", "r")
lines = myfile.read().split('\n')
filtered_urls = []
for url in lines:
    name = url.split('/')[-1]
    if name[0:2] in final_all_countries and name.split('_')[1] in final_pollutants:
        filtered_urls.append(url)
        
with open('filtered_dataset_urls.txt', 'w') as textFile:
    for url in filtered_urls:
        textFile.write("%s\n" % url)
        
#Open filtered dataset urls file and download all datasets in a directory
filteredDatasetFile = open("filtered_dataset_urls.txt", "r")
lines = filteredDatasetFile.read().split('\n')
downfile = urllib.URLopener()
os.chdir('E:/Research/Processing Datasets/Air quality datasets')
for ind, line in enumerate(lines):
    name = line.split('/')[-1]
    try:
        downfile.retrieve(line, name)
        print('Downloaded File ', ind, ':', name)
    except Exception:
        #For any exception like network error, report file, skip processing, pause for 30 seconds and then retry downloading next file
        print('******An exception occured while downloading file [', ind, ':', line, ']...Retrying in 30 seconds: ******')
        time.sleep(30)
        continue

filteredDatasetFile.close()

#Try downloading files failed with exceptionsza
print('Starting to process to download all URL files')
import time
import os
import urllib.request as urllib
os.chdir('E:/Research/Processing Datasets')
start = time.process_time
outputFile = open("Download_dataset_output.txt", "r")
lines = outputFile.read().split('\n')

failedUrls = [line for line in lines if line.startswith('***')]
print(len(failedUrls))
downfile = urllib.URLopener()
os.chdir('E:/Research/Processing Datasets/Air quality datasets')
for ind, line in enumerate(failedUrls):
    try:
        url = line.split(" ")[9]
        name = url.split('/')[-1]
        downfile.retrieve(url, name)
        print('Downloaded File ', ind, ':', name)
    except Exception:
        print('******An exception occured while downloading file [', ind, ':', url, ']...Retrying in 30 seconds: ******')
        time.sleep(30)
        continue

outputFile.close()
print('Processed downloading all files in', time.process_time() - start)